In [ ]:
# Import dependencies
import pandas as pd
import random as random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import linregress
import requests
import json


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
#Jeff Code

#Functions to be used for code

#Function #1
#plotting routine with regression (setting up as a function)
#below function takes 2 sets of x and y data and generates a scatter plot for each.  Then performs a linear regression 
#and returns a dataframe with the regression results
#x and y are number pairs
#data is the name of what is being plotted (like temperature).  used as Y axis label and as part of title
#prints results of regression below graphs

def plotter(x1,y1,Data1,x2,y2,Data2):
   
   #plotting data set x1 and y1

    (slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x1,y1)
    regress_values_N = x1 * slope_N + intercept_N
    line_eq = "y = " + str(round(slope_N,3)) + "x + " + str(round(intercept_N,3))+", r = " +str(round(rvalue_N,3))\
                        + " p = "+str(round(pvalue_N,3))
    max_y = y1.max()
    min_x = x1.min()
    fig = plt.figure(figsize = (15,4))
    ax1 = fig.add_subplot(121)
    ax1.scatter(x1, y1, s=5)
    plt.plot(x1,regress_values_N,"r-")
    plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
    plt.xlabel("Year")
    plt.ylabel(f"{Data1} ")
    plt.title(f"{Data1} vs Year")

  #plotting data set x2 and y2
    (slope_S, intercept_S, rvalue_S, pvalue_S, stderr_S) = linregress(x2,y2)
    regress_values_S = x2 * slope_S + intercept_S
    line_eq = "y = " + str(round(slope_S,3)) + "x + " + str(round(intercept_S,3))+", r = " +str(round(rvalue_S,3))\
                        + " p = "+str(round(pvalue_S,3))
    max_y = y2.max()
    min_x = x2.min()
    #plotting second data set
    ax2 = fig.add_subplot(122)
    ax2.scatter(x2, y2, s=5)
    plt.plot(x2,regress_values_S,"r-")
    plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
    plt.xlabel("Year")
    plt.ylabel(f"{Data2}")
    plt.title(f"{Data2} vs Year")
    
    plt.show()


    Result_Temp_df = pd.DataFrame({"Slope":[slope_N, slope_S], "Intercept":[intercept_N, intercept_S], 
                                                         "r-value":[rvalue_N, rvalue_S], "p-value":[pvalue_N, pvalue_S]})

    Result_Temp_df["Param"] =[Data1, Data2]
    Result_Temp_df = Result_Temp_df[["Param", "Slope", "Intercept", "r-value", "p-value"]]

    print(f"Correlation Results")
    print("")

    print(Result_Temp_df)
    return Result_Temp_df;

In [ ]:
# Kristina's code starts here

In [ ]:
# Kristina's code ends here

In [ ]:
# Jeff's code starts here

#reading in raw data
temp_data_df = pd.read_csv('Resources/clean_temp_data.csv')
temp_data_df.head(3)


In [ ]:
names = temp_data_df.columns
print("Before")
for i in names:
    full = temp_data_df[i].count()
    Null = temp_data_df[i].isnull().sum(axis = 0)
    print(f"for {i} column: Full {full} vs Empty or NaN {Null}")
    


In [ ]:
# Jeff's code ends here


#removing NaN
Start_Length = len(temp_data_df["Date"])
temp_data_df.dropna(inplace = True)

#temp_data_df
Finish_Length = len(temp_data_df["Date"])
print(f"Temperature and precipitation data starts with {Start_Length} records")
print(f"After removing rows with empty data fields have {Finish_Length} records")

#code below combines all locations into one average value for month and year
list = temp_data_df["Name"].unique()
temp_comb_df = temp_data_df.groupby(["Date"])["Total Monthly Precipitation", "Total Montly Snowfall (mm)",
                                             "Avg Monthly Temp", "Monthly Mean Max Temp",
                                              "Monthly Mean Min Temp"].mean().reset_index()

#Data is string Year-Month (month two digits 01-12).  This splits into two fields
temp_comb_df["Month"] = temp_comb_df["Date"].str.split("-").str[1].astype(int)
temp_comb_df["Year"] = temp_comb_df["Date"].str.split("-").str[0].astype(int)

col_list = temp_comb_df.columns.tolist()
Season_data_df = pd.DataFrame(columns = col_list)

#combining months to form seasons: Winter (Dec-Feb), Fall(Mar-May), Summer(June-Aug), Fall(Sept-Nov)
for i in temp_comb_df["Year"].unique():
    if i!= 1950 and i!= 2020: #ignore 1950 because don't have dec 1949 and don't have all 2020
        
        #in dataframe the "month" represents the season: 2=winter, 4 = spring, 7 = summer, 10=fall
        winter_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(1,2)))|\
                                         ((temp_comb_df["Year"]==(i-1)) & (temp_comb_df["Month"]==12))].mean()
        winter_data["Year"]=winter_data["Year"].astype(int)+1 # extra 1 because rounds down to year previous
        winter_data["Month"] = 1 #would be 5 otherwise (mean(12,1,2))
        spring_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(3,5)))].mean()
        spring_data["Year"]=spring_data["Year"].astype(int)
        summer_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(6,8)))].mean()
        summer_data["Year"]=summer_data["Year"].astype(int)
        fall_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(9,11)))].mean() 
        fall_data["Year"]=fall_data["Year"].astype(int)
        
        frames = [winter_data, spring_data, summer_data, fall_data]
        Season_data_df = Season_data_df.append(frames, ignore_index = True)
        
winter = 1
sprint = 4
summer = 7
fall = 10
Season_data_df["Temp Swing"] = Season_data_df["Monthly Mean Max Temp"]-Season_data_df["Monthly Mean Min Temp"]
Season_data_df.drop(columns=["Date"], inplace = True)


In [ ]:
Season_data_df

In [ ]:
variable = "Temp Swing" #pick variable to look at

#defining seasons by value in month column


In [ ]:
Data1 = "Monthly Mean Min Temp"
x1 =  Season_data_df[Season_data_df["Month"]==winter]["Year"]
y1 =  Season_data_df[Season_data_df["Month"]==winter][Data1]
Data2 = "Monthly Mean Max Temp"
x2 =  Season_data_df[Season_data_df["Month"]==winter]["Year"]
y2 =  Season_data_df[Season_data_df["Month"]==winter][Data2]
print("")
print(f"Minimum and Maximum Avg Temperatures for Winter Months")
print("")
Data1 = Data1 + " (Winter)"
Data2= Data2 + " (Winter)"
Correlation_df = plotter(x1,y1,Data1,x2,y2,Data2)

In [ ]:
Data1 = "Temp Swing"
x1 =  Season_data_df[Season_data_df["Month"]==winter]["Year"]
y1 =  Season_data_df[Season_data_df["Month"]==winter][Data1]
Data2 = "Temp Swing"
x2 =  Season_data_df[Season_data_df["Month"]==summer]["Year"]
y2 =  Season_data_df[Season_data_df["Month"]==summer][Data2]
print("")
print(f"Avg Temperature Swings for Winter and Summer (Seasonal max - seasonal min)")
print("")
Data1 = Data1 +" Winter"
Data2 = Data2 +" Summer"
Correlation_df = plotter(x1,y1,Data1,x2,y2,Data2)

In [ ]:
#Analysis for Winter
#chi2 test
#binning data for analysis
year_min = Season_data_df["Year"].min()
year_max = Season_data_df["Year"].max()
delta_year = year_max-year_min
delta_year
bin_size = 4 #number of years

bins = [(year_min+i*bin_size) for i in range(0,int(delta_year/bin_size)+1)]
groups = [((i-1)*4+1953) for i in range(1,int(delta_year/bin_size)+1)]
Season_data_df["Bin"]=pd.cut(Season_data_df["Year"],bins, labels = groups, include_lowest = True)

#Calculating Winter Binning Summary
Season_group_df = Season_data_df[Season_data_df["Month"] == 1].groupby("Bin")["Total Monthly Precipitation", "Total Montly Snowfall (mm)",
                                             "Avg Monthly Temp", "Monthly Mean Max Temp",
                                              "Monthly Mean Min Temp", "Temp Swing"].mean().reset_index()

Param = "Temp Swing"
y_axis = Season_group_df[Param].tolist()
Param_mean = Season_group_df[Param].mean()
Control_list = [Param_mean for i in range(0,len(y_axis))]
# print(Param_mean)
# print(y_axis)
# print(Control_list)

plt.bar(groups, y_axis, color='r', alpha=0.9, align="center")
plt.ylim(min(y_axis)-1, max(y_axis)+1)
plt.ylabel("Temp Swing (Winter) \n 4 year average bin size")
plt.xlabel("Year (Bin)")
plt.title("Bar graph of Temp Swing (winter) versus binned year \n (4 years average together)")

stats.chisquare(y_axis, Control_list)


In [ ]:
#Jeff's code ends here

In [ ]:
# Faduma's code starts here

In [ ]:
# Faduma's code ends here

In [ ]:
# Emily's code starts here

In [ ]:
storm_data = pd.read_csv('Resources/clean_storm_event_data.csv')
storm_data

In [ ]:
storm_data['Event Type'].unique()

# Counts of Tornado Eventsin Minnesota from 1950 to 2020

In [ ]:
year_list = storm_data["Year"].unique()
event_list = storm_data["Event Type"].unique()


clean_tornado_df = pd.DataFrame()
clean_tornado_df = storm_data[storm_data["Event Type"]=="Tornado"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_tornado_df[clean_tornado_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Tornado Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Hail Events in Minnesota from 1950 to 2020

In [ ]:
clean_hail_df = pd.DataFrame()
clean_hail_df = storm_data[storm_data["Event Type"]=="Hail"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_hail_df[clean_hail_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Hail Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Thunderstorm Wind Events in Minnesota from 1950 to 2020

In [ ]:
clean_thunderstorm_wind_df = pd.DataFrame()
clean_thunderstorm_wind_df = storm_data[storm_data["Event Type"]=="Thunderstorm Wind"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_thunderstorm_wind_df[clean_thunderstorm_wind_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Thunderstorm/Wind Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Total Count of Tornado, Thunderstorm/Wind, and Hail Events in Minnesota from 1950 to 2020

In [ ]:
# create dataframe with just tornado, thunderstorm wind, and hail events only
main_events = ['Tornado', 'Thunderstorm Wind', 'Hail']
storm_main_events = storm_data.loc[storm_data['Event Type'].isin(main_events)]
storm_main_events

year_count_df = pd.DataFrame()
for i in year_list:
    year_count = storm_main_events[storm_main_events["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Tornado, Thunderstorm Wind, and Hail Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Number of Tornado, Thunderstorm Wind, and Hail Events per Year

In [ ]:
storm_main_events_graph = storm_main_events
plt.bar(x_data, y_data)
plt.xlabel("Year")
plt.ylabel("Number of Events")
plt.title("Number of Tornado, Thunderstorm Wind, and Hail Events per Year")
plt.show()


In [ ]:
# weather events from 1950-1995 (when only tornadoes, thunderstorm wind, and hail events were recorded)

storm_events_prior_df = storm_data.loc[((storm_data['Year'] >= 1950) & (storm_data['Year'] < 1995))]
storm_events_prior_df

# keeping only year and event type
storm_events_prior_df = storm_events_prior_df[['Year','Event Type']]
storm_events_prior_df

# count_storms = storm_events_prior_df.groupby('Event Type').count()
# count_storms

In [ ]:
# create pie chart for weather events prior to 1995

In [ ]:
# weather events from 1995-2020
later_storm_events_df = storm_data.loc[((storm_data['Year'] > 1995))]
later_storm_events_df

# keeping only year and event type
later_storm_events_df = later_storm_events_df[['Year', 'Event Type']]
later_storm_events_df

In [ ]:
# Emily's code ends here